In [1]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from scipy import stats
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

import Config
from Config import *
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


# Generate data

In [2]:
def randgen_DNA(n, query = "", c_len = None, nMis = 0):
    print("Generating rand seq of length", n)
    
    DNA = "ACGT"
    
    substring = str(query)
    if nMis != 0 and len(substring) >= nMis:
        idxs = np.random.choice(len(substring), size = nMis, replace=False)
        lvls = np.random.randint(1, 4, nMis)
        for i in range(len(idxs)):
            substring = substring[:idxs[i]] + str( (int(substring[idxs[i]]) + lvls[i]) % 4) + substring[idxs[i]+1:]
    substring = str(substring)
    
    m = n - len(substring)
    if m == 0:
        return substring
    datum = [str(np.random.randint(0,4)) for _ in range(m)]
    place = 0
    if c_len is None:
        place = np.random.randint(0,m+1)
    else:
        place = np.random.randint(0,m+1)//c_len * c_len
    string = "".join(datum)
    string = string[:place] + substring + string[place:]
    assert len(string) == n
    assert string.find(substring) != -1
    return string, place


In [3]:
def datagen(n_len, c_len, nMis):
    
    q_len = 2**8
    q_amt = 100
    n_amt = 100

    #n_len = 2**8 + 2 ** 10
    #c_len = 4

    queries = np.asarray([randgen_DNA(q_len) for _ in range(q_amt)])
    queries = queries[:, 0]
    #print(queries[0])

    data = np.asarray([randgen_DNA(n_len, queries[i], c_len, nMis) for i in range(n_amt)])
    refs = data[:,0]
    labs = data[:,1]
    
    q_file = open("query.txt", "w")
    for q in queries:
        q_file.write(q+"\n")
    q_file.close()
    
    r_file = open("ref.txt", "w")
    for r in refs:
        r_file.write(r+"\n")
    r_file.close()
    
    l_file = open("lab.txt", "w")
    for l in labs:
        l_file.write(l+"\n")
    l_file.close()